In [1]:
from sklearn import datasets

# load iris dataset
iris = datasets.load_iris()

In [64]:

import json

def export_network_params(W1, b1, W2, b2, filename=None):
    network_params = {
        "W1": W1.tolist(),
        "b1": b1.tolist(),
        "W2": W2.tolist(),
        "b2": b2.tolist()
    }
    if filename:
        with open(filename, 'w') as file:
            json.dump(network_params, file)
        return ""
    else:
        network_params_json = json.dumps(network_params)
        # Now network_params_json contains the JSON representation of network_params
        return network_params_json


print(type(iris.target))
print(iris.target.size)
print(iris.target[-10:])
print(iris.data[-10:])
print(" ".join(map(str, iris.data[0])))

<class 'numpy.ndarray'>
150
[2 2 2 2 2 2 2 2 2 2]
[[6.7 3.1 5.6 2.4]
 [6.9 3.1 5.1 2.3]
 [5.8 2.7 5.1 1.9]
 [6.8 3.2 5.9 2.3]
 [6.7 3.3 5.7 2.5]
 [6.7 3.  5.2 2.3]
 [6.3 2.5 5.  1.9]
 [6.5 3.  5.2 2. ]
 [6.2 3.4 5.4 2.3]
 [5.9 3.  5.1 1.8]]
5.1 3.5 1.4 0.2


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

'''
# since this is a bunch, create a dataframe
iris_df=pd.DataFrame(iris.data)
iris_df['class']=iris.target
iris_df.columns=['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']
iris_df.dropna(how="all", inplace=True) # remove any empty lines

#view the iris dataframe
print(iris_df)
'''

# Convert to DataFrame for convenience
data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

# Split data into features (X) and labels (y)
X = data.iloc[:, :-1].values
y_val = y = data.iloc[:, -1].values
y = y_val.reshape(-1, 1) # Reshape for one-hot encoding
#print(type(y))
print(y_val.shape, y.shape)
print(y_val[:10])
print(y[:10])

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y)

#print(X[:10])
print(y[:10])
#print(y[0].shape)
#print(X[0].shape)

(150,) (150, 1)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


### Step 3 - Split the data

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)


### Step 4 - Define Neural Network Structure

In [85]:
input_size = X_train.shape[1] # 4 features
hidden_size = 1
output_size = y_train.shape[1] # 3 classes


### Step 5 - Init Parameters

In [86]:
np.random.seed(42) # For reproducibility

W1 = np.random.rand(input_size, hidden_size)
b1 = np.zeros((1, hidden_size))
W2 = np.random.rand(hidden_size, output_size)
b2 = np.zeros((1, output_size))
print(W1.shape)

(4, 1)


### Step 6 - Forward Propagation

In [87]:
def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return expZ / np.sum(expZ, axis=1, keepdims=True)

def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)  # Activation from the hidden layer
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)  # Final output activation
    return A1, A2



### Step 7 - Compute The Loss

In [88]:
def compute_loss(Y, Y_hat):
    """
    Computes the categorical cross-entropy loss.
    
    Parameters:
    - Y: actual labels (one-hot encoded), shape (m, num_classes)
    - Y_hat: predicted probabilities, shape (m, num_classes), from softmax
    
    Returns:
    - loss: the categorical cross-entropy loss
    """
    m = Y.shape[0]
    loss = -np.sum(Y * np.log(Y_hat + 1e-9)) / m # Add a small value to prevent log(0)
    return loss



### Step 8 - Backward Propagation

In [89]:
def backward_propagation_old(X, Y, W1, b1, W2, b2, A1, A2, learning_rate=0.01):
    # Error at output layer
    dZ2 = A2 - Y
    dW2 = np.dot(A1.T, dZ2)
    db2 = np.sum(dZ2, axis=0, keepdims=True)
    
    # Error at hidden layer
    dZ1 = np.dot(dZ2, W2.T) * (1 - np.power(A1, 2))
    dW1 = np.dot(X.T, dZ1)
    db1 = np.sum(dZ1, axis=0, keepdims=True)
    
    # Update parameters
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    
    return W1, b1, W2, b2


def relu_derivative(Z):
    """
    Computes the derivative of the ReLU function.
    Parameters:
    - Z: The input value(s).
    
    Returns:
    - derivative: Gradient of Z where it is positive, and 0 otherwise.
    """
    return (Z > 0).astype(int)

def backward_propagation(X, Y, W1, b1, W2, b2, A1, A2):
    """
    Performs backward propagation and updates the parameters.
    
    Parameters:
    - X: Input data.
    - Y: True labels (one-hot encoded).
    - W1, b1, W2, b2: Parameters of the network.
    - A1, A2: Activations from forward propagation.
    
    Returns:
    - dW1, db1, dW2, db2: Gradients of the loss with respect to W1, b1, W2, b2.
    """
    m = X.shape[0]
    
    # Gradient of loss with respect to Z2
    dZ2 = A2 - Y
    dW2 = (1 / m) * np.dot(A1.T, dZ2)
    db2 = (1 / m) * np.sum(dZ2, axis=0, keepdims=True)
    
    # Backprop into the hidden layer
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(A1)
    dW1 = (1 / m) * np.dot(X.T, dZ1)
    db1 = (1 / m) * np.sum(dZ1, axis=0, keepdims=True)
    
    return dW1, db1, dW2, db2

def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    """
    Updates the parameters using gradient descent.
    
    Parameters:
    - W1, b1, W2, b2: Current parameters.
    - dW1, db1, dW2, db2: Current gradients.
    - learning_rate: Learning rate for the update.
    
    Returns:
    - W1, b1, W2, b2: Updated parameters.
    """
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    
    return W1, b1, W2, b2


### Training

In [120]:
#np.random.seed(17) # For reproducibility

W1 = np.random.rand(input_size, hidden_size)
b1 = np.zeros((1, hidden_size))
W2 = np.random.rand(hidden_size, output_size)
b2 = np.zeros((1, output_size))

In [121]:

learning_rate=0.05
num_epochs = 300
for i in range(num_epochs): # Number of epochs
    A1, A2 = forward_propagation(X_train, W1, b1, W2, b2)
    cost = compute_loss(y_train, A2)
    dW1, db1, dW2, db2 = backward_propagation(X_train, y_train, W1, b1, W2, b2, A1, A2)
    W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
    if i % 100 == 0:
        print(f"Cost after iteration {i}: {cost:.4f}")


Cost after iteration 0: 1.5752
Cost after iteration 100: 1.0015
Cost after iteration 200: 0.6260


In [122]:
j = export_network_params(W1, b1, W2, b2)
print(j)

{"W1": [[0.036058761662048855], [-0.5856192722945736], [0.8556396852569221], [1.1326743771640153]], "b1": [[-0.24287472477262365]], "W2": [[-0.5613963146059882, 0.6609801416890401, 0.9577838116125987]], "b2": [[1.2081190713139576, -0.10388451577294054, -1.1042345555410178]]}


In [123]:
# Make predictions on test data
_, A2_test = forward_propagation(X_test, W1, b1, W2, b2)

# Convert predictions to class labels
predictions = np.argmax(A2_test, axis=1)
print(f"{predictions}")


# Convert actual labels for comparison
actual_labels = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = np.mean(predictions == actual_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")


[0 0 2 1 2 0 2 1 1 1 0 2 2 0 2 2 0 0 2 2 0 0 0 2 2 2 0 1 0 0]
Accuracy: 90.00%


In [124]:
_, A2_train = forward_propagation(X_train, W1, b1, W2, b2)

# Convert predictions to class labels
predictions = np.argmax(A2_train, axis=1)
print(f"{predictions}")


# Convert actual labels for comparison
actual_labels = np.argmax(y_train, axis=1)

# Calculate accuracy
accuracy = np.mean(predictions == actual_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

[1 0 1 1 2 2 2 2 1 2 1 2 1 2 0 2 2 1 2 2 2 1 2 2 0 2 2 0 2 2 1 0 0 2 2 2 0
 0 0 0 0 1 2 2 2 2 2 2 0 1 0 0 2 0 0 2 0 2 2 0 0 2 0 2 2 2 1 2 2 0 2 2 2 2
 0 0 0 2 0 2 0 2 2 2 0 2 2 0 2 0 2 2 0 2 2 2 1 2 0 2 0 2 2 0 2 2 2 2 0 2 2
 2 2 2 0 2 0 0 1 2]
Accuracy: 76.67%


In [119]:
print(actual_labels)


[1 0 1 1 2 2 1 2 1 1 1 2 1 1 0 1 1 1 1 1 1 1 1 2 0 2 2 0 1 2 1 0 0 1 2 2 0
 0 0 0 0 1 2 1 2 2 2 2 0 1 0 0 2 0 0 1 0 1 2 0 0 2 0 2 2 2 1 2 2 0 2 2 1 2
 0 0 0 2 0 2 0 2 1 2 0 1 1 0 2 0 2 1 0 2 1 2 1 1 0 2 0 2 1 0 2 1 2 2 0 2 1
 2 1 1 0 1 0 0 1 1]


In [20]:
X_test[0], y_test[0]

(array([4.8, 3.1, 1.6, 0.2]), array([1., 0., 0.]))